In [3]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.xgboost.model import XGBoostModel
import boto3
from sagemaker import image_uris

In [4]:
sm = boto3.client('sagemaker')
role = get_execution_role()

image_uris.retrieve('xgboost', region='us-east-1', version='1.3-1')
#s3://fraud-model-artifacts/xgb_model_final.tar.gz
s3 = boto3.client('s3')
s3.head_object(Bucket='fraud-model-artifacts', Key='xgb_model_final.tar.gz')
boto3.Session().region_name

container_uri = image_uris.retrieve(
    framework='xgboost',
    region=boto3.Session().region_name,
    version='1.3-1'
)
print(container_uri)



683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.3-1


In [ ]:
sm.create_model(
    ModelName='fraud-xgb-model-flat',
    ExecutionRoleArn=role,
    PrimaryContainer={
        'Image': container_uri,
        'ModelDataUrl': 's3://fraud-model-artifacts/xgb_model_final.tar.gz'
    }
)

In [9]:

sm.create_endpoint_config(
    EndpointConfigName='fraud-xgb-endpoint-config-flat',
    ProductionVariants=[{
        'VariantName': 'AllTraffic',
        'ModelName': 'fraud-xgb-model-flat',
        'InstanceType': 'ml.t2.medium',
        'InitialInstanceCount': 1
    }]
)



{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:869935087425:endpoint-config/fraud-xgb-endpoint-config-flat',
 'ResponseMetadata': {'RequestId': 'a5d9de13-fc80-47f5-8b1a-ee0eb83b83b8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a5d9de13-fc80-47f5-8b1a-ee0eb83b83b8',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '111',
   'date': 'Fri, 27 Jun 2025 02:12:14 GMT'},
  'RetryAttempts': 0}}

In [5]:
sm.create_endpoint(
    EndpointName='fraud-xgb-endpoint-flat',
    EndpointConfigName='fraud-xgb-endpoint-config-flat'
)


{'EndpointArn': 'arn:aws:sagemaker:us-east-1:869935087425:endpoint/fraud-xgb-endpoint-flat',
 'ResponseMetadata': {'RequestId': '26999780-49fb-47ae-8ab6-ac7be0f4d254',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '26999780-49fb-47ae-8ab6-ac7be0f4d254',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '91',
   'date': 'Wed, 09 Jul 2025 17:32:32 GMT'},
  'RetryAttempts': 0}}

In [6]:
sm.describe_model(ModelName='fraud-xgb-model-flat')['PrimaryContainer']['ModelDataUrl']

's3://fraud-model-artifacts/xgb_model_final.tar.gz'

In [10]:
sm.describe_endpoint(EndpointName='fraud-xgb-endpoint-flat')['EndpointStatus']


'InService'